In [1]:
%run PilotForestAreas.ipynb
%run CarbonRevenue.ipynb

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
import param as pm
import panel as pn
import pandas as pd
import hvplot.pandas
pn.extension()

In [3]:
class ForestProject(pm.Parameterized):
    month = pm.Integer(-1, bounds=(-1,None), doc="""
        Represents the number of months in the future that this land is set to be acquired and centree initiated.
        -1 means not scheduled. 0 means starting this month. Positive integer is number of months in future.""")
    forest = pm.ClassSelector(class_=Forest)
    carbon = pm.ClassSelector(class_=Carbon)
    
    def view_table(self):
        return pn.Tabs(("Forest", self.forest.view), ("Carbon", self.carbon.view))

In [4]:
class ForestPipeline(pm.Parameterized):
    projects = pm.List()
    
        
    def __init__(self, projects: list):
        self.projects = [ForestProject(**data) for data in projects]
        
    def view_pipeline(self):
        return pn.Tabs(*[(p.forest.forest_name, p.view_table()) for p in self.projects])
    
    def export_formatted(self):
        df = pd.concat([p.forest.export() for p in self.projects]).T.applymap(Forest._format_numeric)
        return df
    
    def view_export_formatted(self):
        return self.export_formatted().reset_index().hvplot.table()

In [5]:
c1 = Carbon(forest=f1, below_soil_tonnage_factor=0, carbon_price_appreciation=0.15)
c2 = Carbon(forest=f2, below_soil_tonnage_factor=0, carbon_price_appreciation=0.15)


forest_project_data = [
    {'month': 8, 'forest': f1, 'carbon': c1},
    {'month': 15, 'forest': f2, 'carbon': c2},
]

In [6]:
forest_pipeline = ForestPipeline(forest_project_data)

In [7]:
p = forest_pipeline.view_pipeline()

In [8]:
forest_pipeline.view_pipeline()

Tabs
    [0] Tabs
        [0] ParamMethod(method, _pane=Row, name='Forest')
        [1] ParamMethod(method, _pane=Row, name='Carbon')
    [1] Tabs
        [0] ParamMethod(method, _pane=Row, name='Forest')
        [1] ParamMethod(method, _pane=Row, name='Carbon')

In [10]:
forest_pipeline.view_export_formatted()

:Table   [Field,Type,('KNF1',),('KNF2',)]